In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_data = train_data.drop(["PassengerId", "Name", "Ticket"], axis =1 )
test_data = test_data.drop(["Name","Ticket"],axis = 1)

In [5]:
train_data.isnull().sum()
test_data.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
Age             86
SibSp            0
Parch            0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [6]:
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
test_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())

train_data["Fare"] = train_data["Fare"].fillna(train_data["Fare"].median())
test_data["Fare"] = train_data["Fare"].fillna(train_data["Fare"].median())

train_data = train_data.drop(columns=["Cabin"])
test_data = test_data.drop(columns=["Cabin"])

In [7]:
x = train_data.drop(columns = ["Survived"])
y = train_data["Survived"]


In [8]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(x , columns = ["Sex","Embarked"])
test_data = pd.get_dummies(test_data, columns = ["Sex","Embarked"])
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size= 0.2,random_state = 0)

In [9]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100,random_state = 0)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

In [10]:
from sklearn.metrics import confusion_matrix,classification_report

cs = confusion_matrix(y_pred,y_test)
print(classification_report(y_pred,y_test))
print(cs)

              precision    recall  f1-score   support

           0       0.90      0.85      0.87       117
           1       0.74      0.82      0.78        62

    accuracy                           0.84       179
   macro avg       0.82      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179

[[99 18]
 [11 51]]


In [11]:
import xgboost as xgb
model = xgb.XGBClassifier(use_label_encoder = False,eval_metric = "logloss")
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

cs_xgb = confusion_matrix(y_pred,y_test)
print(classification_report(y_pred,y_test))
print(cs_xgb)

              precision    recall  f1-score   support

           0       0.93      0.85      0.89       120
           1       0.74      0.86      0.80        59

    accuracy                           0.85       179
   macro avg       0.83      0.86      0.84       179
weighted avg       0.87      0.85      0.86       179

[[102  18]
 [  8  51]]


In [12]:
test_data1 = test_data.drop(columns=["PassengerId"])
test_preds = model.predict(test_data1)

submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_preds
})

submission.to_csv("submission.csv",index = False)